# Assembler

In [1]:
def preprocess_assembly_code(assembly_code):
    assembly_code = assembly_code.splitlines()

    line_no=0
    line_alias={}

    #Line alias
    for line in assembly_code:
        s = line.strip().split(' ')
        if not s[0]:
            continue
        if s[0][-1]==':':
            line_alias[s[0][0:-1]]=line_no

        line_no+=1
    # print("line_alias:   ",line_alias, '\n')

    #Preprocessing (Removing alias)
    preprocessd_assembly_code = []
    for line in assembly_code:
        s = line.strip().split(' ')
        if not s[0]:
            continue
        if s[0][-1]==':':
            s=s[1:]
        # print(s)        

        inst = s[0]
        if inst == 'IF_P' or inst == 'ELSE_P':     #IF_P or ELSE_P
            out= [inst , line_alias[s[1]] ]
        else:
            out = s

        preprocessd_assembly_code.append(out)

    return preprocessd_assembly_code

# preprocess_assembly_code(assembly_code)

In [2]:
class SPCore():
    def __init__(self, THRD_ID):
        # self.R = [0 for i in range(16)]
        self.R={
            'R0'    :0,'R1'    :0,'R2'    :0,'R3'    :0,'R4'    :0,'R5'    :0,'R6'    :0,'R7'    :0,                'R8'    :0,'R9'    :0,'R10'    :0,'R11'    :0,'R12'    :0,'R13'    :0,'R14'    :0,
            'R15'    :0
        }
        self.THRD_ID = THRD_ID
        self.P = True
        self.P_stack = [True]

    def __str__(self):
        return (
            "SPCore:\t"+str(self.THRD_ID)+'\n'+
            "P\t"+str(self.P)+'\n'+
            "P_stack\t"+str(self.P_stack)+'\n'+
            str(self.R)+'\n'
        )


    def pushMask(self):
        self.P_stack.append(self.P)
    def popMask(self):
        self.P_stack.pop()
    def curMask(self):
        return self.P_stack[-1]
    def invMask(self):
        self.P_stack[-1] = not self.P_stack[-1]


##############################
def all_p_false(cores):
    for core in cores:
        if core.curMask()==True:
            return False
    return True

def all_p_true(cores):
    for core in cores:
        if core.curMask()==False:
            return False
    return True

def valid(cores):
    return [core for core in cores if core.curMask()==True ]

def pushStack(cores): #TODO Consider 'X' state
    for core in cores:
        core.pushMask()

def compStack(cores): #TODO Consider 'X' state
    for core in cores:
        core.invMask()

def popStack(cores):
    for core in cores:
        core.popMask()    

In [ ]:
# class SMCores():
#     def __init__(self):

In [3]:
class Scheduler():
    def __init__(self,f_path, cores):
        self.assembly_code = open(f_path+"/prog.txt", "r").read()
        self.assembly_code = preprocess_assembly_code(self.assembly_code)

        self.DataMem = list(map(int,open(f_path+"/data.txt", "r").read().splitlines()))

        self.cores = cores

        print(self.assembly_code, '\n')
        # print(len(self.assembly_code), '\n')
        self.PC = 0

    def runProgram(self):        
        while self.PC <len(self.assembly_code):
            self.execute(self.assembly_code[self.PC])
            print('PC',self.PC)
            print(self.cores[0])
        print('\n____Finished___')

    def execute(self, s):
        # print(s)
        opcode = s[0]
        print(s)
        print('valid',len(valid(self.cores)))

        #Normal Instructions - Runs only on valid (mask==1) cores
        if opcode == 'LOAD':
            x=s[1]
            y=s[2]
            # self.R[x]=DataMem[self.R[y]]
            for core in valid(self.cores):
                core.R[x]=self.DataMem[core.R[y]] #TODO check mask
        elif opcode == 'LOADI':
            x=s[1]
            I=int(s[2])
            # self.R[x]=I
            for core in valid(self.cores):
                core.R[x]=I
        elif opcode == 'LOADC':
            x=s[1]
            c=s[2]
            if c=='THRD_ID': #TODO:add rest
                # self.R[x]=self.THRD_ID
                for core in valid(self.cores):
                    core.R[x]=core.THRD_ID
        elif opcode == 'STORE':
            x=s[1]
            y=s[2]
            # DataMem[self.R[y]] = self.R[x]
            for core in valid(self.cores):
                self.DataMem[core.R[y]] = core.R[x]
        elif opcode == 'SETP':
            op = s[1]
            x = s[2] 
            y = s[3]
            if op=='EQ': #TODO: Add rest
                for core in valid(self.cores):
                    core.P = (core.R[x]==core.R[y])


        #Branching Instructions
        elif opcode == 'IF_P':
            L = int(s[1])
            pushStack(self.cores) #TODO Check prev mask
            if all_p_false(self.cores):
                self.PC = L
                return


        elif opcode == 'ELSE_P':
            L = int(s[1])
            compStack(self.cores) #TODO ignote 'X' mask
            if all_p_true(self.cores):
                self.PC = L
                return

        elif opcode == 'ENDIF':
            popStack(self.cores)

        else:
            raise Exception("Unknown Instruction")


        self.PC+=1
        return

In [4]:
# f_path = 'sample/1-simple-branch'
f_path = 'sample/2-nested-branch'

gpu = Scheduler(f_path, cores = [
    SPCore(0), SPCore(1), SPCore(2), SPCore(3)
])

gpu.runProgram()

print(gpu.DataMem)

[['LOADC', 'R0', 'THRD_ID'], ['LOAD', 'R5', 'R0'], ['LOADI', 'R6', '6'], ['SETP', 'EQ', 'R5', 'R6'], ['IF_P', 7], ['LOADI', 'R5', '10'], ['ELSE_P', 8], ['LOADI', 'R5', '20'], ['ENDIF'], ['STORE', 'R5', 'R0']] 

['LOADC', 'R0', 'THRD_ID']
valid 4
PC 1
SPCore:	0
P	True
P_stack	[True]
{'R0': 0, 'R1': 0, 'R2': 0, 'R3': 0, 'R4': 0, 'R5': 0, 'R6': 0, 'R7': 0, 'R8': 0, 'R9': 0, 'R10': 0, 'R11': 0, 'R12': 0, 'R13': 0, 'R14': 0, 'R15': 0}

['LOAD', 'R5', 'R0']
valid 4
PC 2
SPCore:	0
P	True
P_stack	[True]
{'R0': 0, 'R1': 0, 'R2': 0, 'R3': 0, 'R4': 0, 'R5': 5, 'R6': 0, 'R7': 0, 'R8': 0, 'R9': 0, 'R10': 0, 'R11': 0, 'R12': 0, 'R13': 0, 'R14': 0, 'R15': 0}

['LOADI', 'R6', '6']
valid 4
PC 3
SPCore:	0
P	True
P_stack	[True]
{'R0': 0, 'R1': 0, 'R2': 0, 'R3': 0, 'R4': 0, 'R5': 5, 'R6': 6, 'R7': 0, 'R8': 0, 'R9': 0, 'R10': 0, 'R11': 0, 'R12': 0, 'R13': 0, 'R14': 0, 'R15': 0}

['SETP', 'EQ', 'R5', 'R6']
valid 4
PC 4
SPCore:	0
P	False
P_stack	[True]
{'R0': 0, 'R1': 0, 'R2': 0, 'R3': 0, 'R4': 0, 'R5': 5, '